#### Postponed annotations

> Both postponed annotations via the future import and `ForwardRef` require Python 3.7+.

Postponed annotations (as described in `PEP563`) "just work".

In [1]:
from __future__ import annotations
from typing import Any, List, ForwardRef
from pydantic import BaseModel, HttpUrl
from pydantic.errors import ConfigError

In [2]:
class Model(BaseModel):
    a: List[int]
    b: Any

In [3]:
print(Model(a=("1", 2, 3), b="ok"))

a=[1, 2, 3] b='ok'


Internally, pydantic will call a method similar to `typing.get_type_hints` to resolve annotations.

In cases where the referenced type is not yet defined, `ForwardRef` can be used (although referencing the type directly or by its string is a simpler solution in the case of `self-referencing models`).

In some cases, a `ForwardRef` won't be able to be resolved during model creation. For example, this happens whenever a model references itself as a field type. When this happens, you'll need to call `update_forward_refs` after the model has been created before it can be used:

In [4]:
Foo = ForwardRef("Foo")

In [5]:
class Foo(BaseModel):
    a: int = 123
    b: Foo = None

In [6]:
Foo.update_forward_refs()

In [7]:
print(Foo())

a=123 b=None


In [8]:
print(Foo(b={"a": "321"}))

a=123 b=Foo(a=321, b=None)


> To resolve strings (type names) into annotations (types), pydantic needs a namespace dict in which to perform the lookup. For this it uses `module.__dict__`, just like `get_type_hints`. This means pydantic may not play well with types not defined in the global scope of a module.

For example, this works fine:

In [9]:
def this_works():
    class Model(BaseModel):
        a: HttpUrl

    print(Model(a="https://example.com"))

In [10]:
this_works()

a=HttpUrl('https://example.com', )


While this will break:

In [11]:
def this_is_broken():
    from pydantic import HttpUrl  # HttpUrl is defined in function local scope

    class Model(BaseModel):
        a: HttpUrl

    try:
        Model(a="https://example.com")
    except ConfigError as e:
        print(e)

    try:
        Model.update_forward_refs()
    except NameError as e:
        print(e)

In [12]:
this_is_broken()

Resolving this is beyond the call for pydantic: either remove the future import or declare the types globally.